In [ ]:
from pycalphad import Database, equilibrium, variables as v, calculate
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)

In [2]:
# constants:
P = 101325

# Компоненты и фазы

In [3]:
import random

random.seed(10)
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'PM', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()
print('Количество фаз =' + str(len(all_phases)))
print('Количество компонент =' + str(len(all_components)))
print(all_components)
# components = random.sample(all_components, 8)
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']

print(components)


Количество фаз =2023
Количество компонент =56
['AG', 'AL', 'AU', 'BA', 'BE', 'BI', 'C', 'CA', 'CD', 'CE', 'CO', 'CR', 'CU', 'DY', 'ER', 'EU', 'FE', 'GA', 'GD', 'HF', 'HG', 'HO', 'IN', 'IR', 'K', 'LA', 'LI', 'MG', 'MN', 'MO', 'NA', 'NB', 'ND', 'NI', 'OS', 'PB', 'PD', 'PM', 'PR', 'PT', 'RE', 'RH', 'SC', 'SI', 'SM', 'SN', 'TA', 'TB', 'TC', 'TI', 'TL', 'V', 'W', 'Y', 'ZN', 'ZR']
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']


# Бинарные фазы для компонент

In [4]:
counter = 0 
possible_phases = []

for phase in db.phases.items():
    # item[0] - name, item[1] - content
    # item[1] - name, constituents, sublattices, model_hints    
    found_sublattices = 0
    for sublattice_c in phase[1].constituents:
        found = False
        for comp in components:
            if len([species.name for species in sublattice_c if species.name == comp]) > 0:
                found = True
                break
        if found:
            found_sublattices += 1
        else:
            break
    
    if found_sublattices == len(phase[1].constituents):
        possible_phases.append(phase[0])
        
possible_phases.sort()
print(possible_phases)

['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 'RHOMBO_C19', 'SIGMA', 'TETRAGONAL_A6',

In [5]:
import numpy as np

def getPhases(int_mask, all_phases = all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]
def phasesToMask(phases, all_phases = all_phases):
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)
def phaseToMask(phase, all_phases = all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

phases_mask = phasesToMask(possible_phases)
print(possible_phases == getPhases(phases_mask, all_phases))

True


# Расчет фазовых равновесий

In [6]:
skip_cond_component = 'V'
temp = 1200
conditions = {v.T:temp,v.P:P}
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']
# TODO: make random amount of component
amount = 1.0/len(components)

for comp in components:
    if comp != skip_cond_component:
        conditions[v.X(comp)] = amount

calc_components = components
calc_components.append('VA')
print(conditions)
print(calc_components)
eq = equilibrium(db,calc_components,getPhases(phases_mask),conditions, verbose=False)
# gibbs_energy = eq.GM

{T: 1200, P: 101325, X_AU: 0.125, X_RH: 0.125, X_SM: 0.125, X_ZR: 0.125, X_AL: 0.125, X_HG: 0.125, X_SE: 0.125}
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE', 'VA']


In [7]:
def flatten_array(arr):
    result = []
    for item in arr.flat:
        if isinstance(item, np.ndarray):
            result.extend(flatten_array(item))
        else:
            result.append(item)
    return result

In [8]:
eq_phases = flatten_array(np.array(eq.Phase))
print(eq_phases)
eq_phases_bin = [phaseToMask(phase) for phase in eq_phases]
print([getPhases(phase) for phase in eq_phases_bin])

['AUZR', 'LIQUID', 'AL2SM', '', '', '', '', '', '']
[['AUZR'], ['LIQUID'], ['AL2SM'], [], [], [], [], [], []]


# Функции

In [9]:
from pycalphad.core.utils import instantiate_models
# instantiate_models(db, components, getAllPhases(components_phases))
print(getPhases(phases_mask))

['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 'RHOMBO_C19', 'SIGMA', 'TETRAGONAL_A6',

In [57]:
from symengine import Piecewise, Symbol, lib
import sympy

def parseMul(mul):
    key = ""
    value = 1
    for arg in mul:
        if isinstance(arg, sympy.core.numbers.NegativeOne) or isinstance(arg, sympy.core.numbers.Float):
            value *= arg
        else:
            key += str(arg)
    return (key, value)
def addKey(dict_, key, value):
    if key in dict_:
        dict_[key] += value
    else:
        dict_[key] = value

def parsePiecewise(expr, temp, parsed_dict = {}, multiplier = 1):
    argument = 0
    T = sympy.Symbol('T')
    for arg in expr.args_as_sympy():
        if not arg.is_Boolean and not isinstance(arg, sympy.core.Rel):
            argument = arg.as_coeff_Mul()
        elif arg.subs(T,temp):
            break
    if argument == 0:
        raise RuntimeError(f"Failed to parse:{expr}")
    # print(argument)
    
    multiplier *= argument[0]
    arguments = argument[1].args
    if not isinstance(argument[1], sympy.core.add.Add):
        arguments= [argument[1]]
    for arg in arguments:
        ordered_arg = arg.as_ordered_factors()
        if len(ordered_arg) == 1:
            element = ordered_arg[0]
            if isinstance(element, sympy.core.symbol.Symbol):
                element = str(element)
                if element in db.symbols.keys():
                    parsed_dict = parsePiecewise(db.symbols[element], temp, parsed_dict, multiplier)
                else:
                    key = element
                    addKey(parsed_dict, key, multiplier)
            else:
                key = "T**0"
                addKey(parsed_dict, key, element * multiplier)
        elif len(ordered_arg) > 1:           
            key, value = parseMul(ordered_arg)
            if key in db.symbols.keys():
                parsed_dict = parsePiecewise(db.symbols[key], temp, parsed_dict, value*multiplier)
            else:
                addKey(parsed_dict, key, value * multiplier)
    
    return parsed_dict

expr = db.symbols['GHSERZR']
parsePiecewise(expr, temp)

{'T**0': -7827.59500000000,
 'T**(-1.0)': 34971.0000000000,
 'T': 125.649050000000,
 'T**2.0': -0.00437791000000000,
 'Tlog(T)': -24.1618000000000}

In [11]:
'GHSERAU' in db.symbols.keys()

True

In [12]:
phase_param = {}
for s in db._parameters:
    phase_name = s['phase_name']
    func = s['parameter']
    species = s['constituent_array']
    if phase_name in phase_param:
        phase_param[phase_name] += [species, func]
    else:
        phase_param[phase_name] = [species, func]

print(phase_param)

{'FCC_A1': [((Species('AG', 'AG1'),), (Species('VA', 'VA1'),)), Piecewise((GHSERAG, And(T < 3000.0, 298.15 <= T)), (0, True)), ((Species('AL', 'AL1'),), (Species('VA', 'VA1'),)), Piecewise((GHSERAL, And(T < 2900.0, 298.15 <= T)), (0, True)), ((Species('AM', 'AM1'),), (Species('VA', 'VA1'),)), Piecewise((GFCCAM, And(T < 3000.0, 298.15 <= T)), (0, True)), ((Species('AS', 'AS1'),), (Species('VA', 'VA1'),)), Piecewise((GFCCAS, And(T < 1200.0, 298.15 <= T)), (0, True)), ((Species('AU', 'AU1'),), (Species('VA', 'VA1'),)), Piecewise((GHSERAU, And(T < 3200.0, 298.15 <= T)), (0, True)), ((Species('BA', 'BA1'),), (Species('VA', 'VA1'),)), Piecewise((GFCCBA, And(T < 4000.0, 298.15 <= T)), (0, True)), ((Species('BE', 'BE1'),), (Species('VA', 'VA1'),)), Piecewise((GFCCBE, And(T < 3000.0, 298.15 <= T)), (0, True)), ((Species('BI', 'BI1'),), (Species('VA', 'VA1'),)), Piecewise((GFCCBI, And(T < 3000.0, 298.15 <= T)), (0, True)), ((Species('CA', 'CA1'),), (Species('VA', 'VA1'),)), Piecewise((GHSERCA, A

In [13]:
print(phase_param["AUZR"])
print(parsePiecewise(phase_param["AUZR"][1], temp))

[((Species('AU', 'AU1'),), (Species('ZR', 'ZR1'),)), Piecewise((-119646.3 + GHSERAU + GHSERZR + 12.5555*T, And(T < 6000.0, 298.15 <= T)), (0, True))]
{'T**0': -228887.971000000, 'T**(-1.0)': 10707152.0000000, 'T': 1285.54903000000, 'T**2.0': 0.0788043300000000, 'Tlog(T)': -204.030344900000, 'T**3.0': -1.15187130000000e-5}


# Текущие результаты:

In [14]:
print('Все фазы для компонентов из списка:')
print(len(getPhases(phases_mask)))
print(getPhases(phases_mask))

Все фазы для компонентов из списка:
93
['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 

In [15]:
print('Все фазы для компонентов из списка:')
print(len(getPhases(phases_mask)))
print(getPhases(phases_mask))

Все фазы для компонентов из списка:
93
['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 

In [16]:
print('Все фазы для компонентов из списка:')
print(len(getPhases(phases_mask)))
print(getPhases(phases_mask))

Все фазы для компонентов из списка:
93
['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 

In [17]:
print(f'Все фазы для компонентов из списка(бинарные): {bin(phases_mask)}')

Все фазы для компонентов из списка(бинарные): 0b1000010000000000000001000000000000000001100000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000001100000000000000011000000000000000000001000000000100000000000000000000000000000000000000000000010000000000000000000000000000000000000110000000000000000000000000000000000100000000000001000000000000000000000000000000000000000100000000000000000000000000000000000000000000001100000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000010000000000000000000000100000000000000000000000000111000000000000000000000000000000001100010000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000001000000100000001000000000000000000000000000000000110000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [18]:
print('Параметры расчета:')
print(f'Давление {P}, Температура {temp}, Количестов вещества по {amount} для компоненты')

Параметры расчета:
Давление 101325, Температура 1200, Количестов вещества по 0.125 для компоненты


In [77]:
import pandas as pd

print('Значения функций для фаз:')
unable_phases = ['FCC_L10', 'BIIN_EPSILON', '']
df = pd.DataFrame()

for phase in all_phases:
    if phase not in unable_phases:
        species = {}
        for i in range(len(phase_param[phase])):
            if i % 2 == 1:
                # print(f'{phase}({int(i/2)})' )
                parsed_piecewise = parsePiecewise(phase_param[phase][i], temp, {}, 1)
                parsed_piecewise['phase'] = phase
                parsed_piecewise = {**parsed_piecewise, **species}
                df = df.append(parsed_piecewise, ignore_index=True)
                # print(f'{phase}({int(i/2)}) = {parsed_piecewise}')
            else:
                for j in range(len(phase_param[phase][i])):
                    species[f"S{j+1}"] = phase_param[phase][i][j][0]
df

Значения функций для фаз:


,T**0,T**(-1.0),T,T**2.0,T**3.0,Tlog(T),phase,S1,S2,T**(-9.0),T**4.0,S3,S4,T**7.0,T**(-3.0),T**(-2.0),S5,T**(-5.0),T**(-7.0),Tlog(1.0e-5*P)
0,-31495.0000000000,36411.0000000000,157.890000000000,0.00200100000000000,-1.27200000000000e-6,-27.7440000000000,A15_CR3PT,CR,PT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1195.36100000000,NaN,183.871136000000,NaN,NaN,-31.7481920000000,A2_BCC,AL,VA,-1.23062200000000e+28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,160000.000000000,NaN,-31.0000000000000,NaN,NaN,NaN,A2_BCC,AL,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3248.24100000000,72527.0000000000,132.652210000000,-0.00265473900000000,-1.73480000000000e-7,-25.0861000000000,A2_BCC,CO,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1450.00000000000,NaN,NaN,NaN,NaN,NaN,A2_BCC,CO,VA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12574,-457478.483000000,21014211.0000000,2324.25937800000,-0.0139549900000000,-3.06720000000000e-7,-343.418200000000,ZRB12,ZR,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12575,-194827.769000000,1783241.00000000,324.484794000000,-0.00517600000000000,-2.55600000000000e-8,-50.7665000000000,ZRB_B1,ZR,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12576,-131570.490000000,34971.0000000000,367.648830000000,-0.0116958000000000,-4.87890000000000e-7,-73.2102500000000,ZRPB2,ZR1PB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12577,-182203.400000000,148517.000000000,258.514540000000,-0.00439386500000000,5.50000000000000e-11,-45.1863100000000,ZRSI,SI,ZR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df.to_csv("./ParsedData/functions.csv")

In [79]:
print('Фазы для заданых параметров:')
print(eq_phases)
print('В бинарном виде:')
print([bin(phase) for phase in eq_phases_bin])

Фазы для заданых параметров:
['AUZR', 'LIQUID', 'AL2SM', '', '', '', '', '', '']
В бинарном виде:
['0b10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000', '0b10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000